In [ ]:
from tensorflow.python.keras import models
from tensorflow.keras import layers
from tensorflow.python.keras import losses
from tensorflow.keras import optimizers
from tensorflow.python.keras import activations
from tensorflow.python.keras import initializers
from tensorflow.python.keras import backend
from tensorflow.python.keras import metrics
from tensorflow.python import keras
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
test_x_rc = np.load('')
test_x_other = np.load('')

In [ ]:

class ExpandTileLayer(layers.Layer):
    def __init__(self):
        super(ExpandTileLayer, self).__init__()
    def call(self, inputs, *args, **kwargs):
        return backend.tile(backend.expand_dims(inputs, axis=-2), (1, 80, 1))

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():


    #%%

    rc_input = keras.Input(shape=(test_x_rc.shape[-1], ),
                           dtype='int32',
                           name='rc_input_layer')
    other_x_input = keras.Input(shape=(80, test_x_other.shape[-1]),
                                dtype='float32',
                                name='other_x_input')
    rc_embedding_layer = layers.Dense(units=32,
                                      use_bias=False, name='rc_embed_layer')
    rc_embedding = rc_embedding_layer(rc_input)

    # 维度拓展
    rc_embedding = ExpandTileLayer()(rc_embedding)

    x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
    ox = layers.Bidirectional(layers.LSTM(units=128,
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(x_input)

    ox = layers.Bidirectional(layers.LSTM(units=64,
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(ox)
    ox = layers.Dense(units=128,
                      kernel_initializer=initializers.initializers_v2.GlorotUniform())(ox)
    # if hp.Choice('d2_activation', ['prelu', 'elu']) == 'prelu':
    lstm_output = layers.ELU()(ox)
    # else:
    #     lstm_output = layers.ELU()(ox)

    conv1d_1 = layers.Conv1D(filters=256,
                             kernel_size=11,
                             padding='same', use_bias=False)(x_input)
    conv1d_1 = layers.BatchNormalization()(conv1d_1)
    conv1d_1 = layers.ReLU()(conv1d_1)
    conv1d_2 = layers.Conv1D(filters=128, padding='same', use_bias=False,
                             kernel_size=5)(conv1d_1)
    conv1d_2 = layers.BatchNormalization()(conv1d_2)
    conv1d_output = layers.ReLU()(conv1d_2)

    output = layers.Concatenate(axis=-1)([lstm_output, conv1d_output])
    output = layers.Dense(units=256,
                          activation=activations.relu,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)
    output = layers.Dense(units=1,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)

    my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[output])

    # my_callbacks = [callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, verbose=1),
    #                 callbacks.EarlyStopping(monitor='val_loss', patience=46, verbose=1, mode='min', restore_best_weights=True)]
    my_model.compile(loss=losses.MeanAbsoluteError(),
                     optimizer=optimizers.Adam(),
                     metrics=[metrics.MeanAbsoluteError()])

In [ ]:
my_model.load_weights('')
pre_y = my_model.predict(x=[test_x_rc, test_x_other], batch_size=1024)
pre_y = np.array(pre_y).flatten()
pre_y = np.vstack([np.arange(1, pre_y.shape[0] + 1), pre_y])
pre_y = np.transpose(pre_y, axes=(1, 0))
pre_y = pd.DataFrame(pre_y, columns=['id', 'pressure'])
pre_y.to_csv('./submission.csv', index=False)

